In [2]:
import pandas as pd
from pathlib import Path
X_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_train.csv"),sep=r"\s+")
Y_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_train.csv"),header=None, names=['category'])
X_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_test.csv"),sep=r"\s+")
Y_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_test.csv"),header=None, names=['category'])
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1668, 9635) (1668, 1) (557, 9635) (557, 1)


In [3]:
XY_train = pd.concat([X_train, Y_train], axis=1)
print(XY_train.shape)
XY_train.head()

(1668, 9636)


,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour,category
0,1,1,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
word_counts = XY_train.groupby('category').agg('sum')
print(word_counts.head())

           ad  sale  boost  time  warner  profit  quarterli  media  giant  \
category                                                                    
0         125   284     87   164       0     187          9     23     69   
1          88    95     12   159      10       7          0     31     12   
2         142     8     34   239       0       3          1     27      3   
3         157    34     16   262       0       0          0     19      5   
4         119    91     17   284      10      18          2    145     45   

          jump  ...  Â£339  denialofservic  ddo  seagrav  bot  wirelessli  \
category        ...                                                         
0           28  ...      0               0    0        0    0           0   
1            4  ...      0               0    0        0    0           0   
2            3  ...      0               0    0        0    0           0   
3           40  ...      0               0    0        0    0           0  

In [5]:
total_words_per_category = word_counts.sum(axis=1)
print(total_words_per_category)

category
0    63889
1    48010
2    71554
3    62439
4    78433
dtype: int64


In [6]:
word_probs = word_counts.div(total_words_per_category, axis=0)
word_probs.reset_index(inplace=True)
print(word_probs.head())

   category        ad      sale     boost      time    warner    profit  \
0         0  0.001957  0.004445  0.001362  0.002567  0.000000  0.002927   
1         1  0.001833  0.001979  0.000250  0.003312  0.000208  0.000146   
2         2  0.001985  0.000112  0.000475  0.003340  0.000000  0.000042   
3         3  0.002514  0.000545  0.000256  0.004196  0.000000  0.000000   
4         4  0.001517  0.001160  0.000217  0.003621  0.000127  0.000229   

   quarterli     media     giant  ...     Â£339  denialofservic       ddo  \
0   0.000141  0.000360  0.001080  ...  0.000000        0.000000  0.000000   
1   0.000000  0.000646  0.000250  ...  0.000000        0.000000  0.000000   
2   0.000014  0.000377  0.000042  ...  0.000000        0.000000  0.000000   
3   0.000000  0.000304  0.000080  ...  0.000000        0.000000  0.000000   
4   0.000025  0.001849  0.000574  ...  0.000025        0.000038  0.000115   

   seagrav      bot  wirelessli  streamcast  peripher  headphon   flavour  
0  0.00000

In [7]:
word_probs.sum(axis=1)


0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

In [8]:
priors = Y_train['category'].value_counts().sort_index() / len(Y_train)
priors = priors.reset_index()
priors.columns = ['category', 'probability']
print(priors)
print(priors.sum())

   category  probability
0         0     0.224820
1         1     0.170264
2         2     0.191247
3         3     0.226019
4         4     0.187650
category       10.0
probability     1.0
dtype: float64


In [9]:
X_test.head()

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,Â£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,6,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
import numpy as np
L = word_probs.to_numpy()
L = np.log(L[:, 1:])  # Exclude the 'category' column
print(L)

[[ -6.23658874  -5.41592824  -6.59899436 ...         -inf         -inf
          -inf]
 [ -6.30182779  -6.22528771  -8.29425795 ...         -inf         -inf
          -inf]
 [ -6.22238063  -9.09876615  -7.65184716 ...         -inf         -inf
          -inf]
 [ -5.98569955  -7.51558483  -8.26935664 ...         -inf         -inf
          -inf]
 [ -6.49087654  -6.75914053  -8.43678669 ...  -9.88370568 -10.17138775
  -10.57685286]]


C:\Users\90553\AppData\Local\Temp\ipykernel_3132\3195139691.py:3: RuntimeWarning: divide by zero encountered in log
  L = np.log(L[:, 1:])  # Exclude the 'category' column


In [11]:
P = priors['probability'].to_numpy()
print(P)
P = np.log(P)
print(P)

[0.22482014 0.17026379 0.191247   0.22601918 0.18764988]
[-1.49245456 -1.77040634 -1.65418948 -1.4871354  -1.67317739]


In [12]:
X = X_train.to_numpy()
print(X)

[[1 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [13]:
M = np.zeros((X.shape[0], P.shape[0]))  # Initialize M with zeros
priorFlag = np.zeros(M.shape[0])  # Initialize priorFlag
for i in range(M.shape[0]):  # for each doc i
    for j in range(X.shape[1]):  # For each word j

        for k in range(L.shape[0]):  # For each category
            if(priorFlag[i]==0):
                M[i, k] += P[k] # Add prior only once per document
                priorFlag[i]=1
            M[i,k] += L[k, j] * X[i, j] # Add log-likelihood multiplied by word freq
print(M)


C:\Users\90553\AppData\Local\Temp\ipykernel_3132\1181685707.py:10: RuntimeWarning: invalid value encountered in scalar multiply
  M[i,k] += L[k, j] * X[i, j] # Add log-likelihood multiplied by word freq


KeyboardInterrupt: 